# 贪心算法
- 贪心算法（又称贪婪算法）是指，在对问题求解时，总是做出在当前看来是最好的选择。也就是说，不从整体最优上加以考虑，每一步都是着眼于当前情况下的最优解，但最终的全局最优解是通过局部最优解迭代得到的。
- 贪心算法能否得到全局最优解的关键是关贪心策略(度量标准)的选择，使用的贪心策略必须具备无后效性，即某个状态以前的过程不会影响以后的状态，只与当前状态有关。

## 背包问题
- 问题描述如下，有一个空间容量为V的背包，有一些可以装进背包的物品，每个物品都有相应的价值w[i]与占据的空间v[i]，这些物品都可以被任意分割为几部分，问该背包所能装载物品的最大价值是多少？
- 使用贪心解法，其中最为关键的度量标准是 **单位体积的物品价值**，即尽可能多的优先选择 **单位体积的物品价值** 更高的物品放进背包

In [1]:
from __future__ import division

In [2]:
def backpack(V, w, v):
    """
    type V:float
    type w:[int]
    type v:[int]
    rtype :float
    """
    
    # step 1, 将物品价值-体积组合起来，方便排序
    w_v = zip(w, v)
    
    # step 2, 按照每件物品的 单位体积的价值 对其进行降序排序
    w_v.sort(key = lambda item: item[0]/item[1] ,reverse = True)
    
    # step 3, 按照度量标准选择物品
    W = 0 # 空背包的价值
    for i in range(len(w_v)):
        if V == 0:         # 背包没有空间
            break
        if w_v[i][1] < V:  # 背包剩余空间能完全装下当前的物品
            W += w_v[i][0]
            V -= w_v[i][1]
        else:           # 背包剩余空间不能完全装下当前的物品
            W += w_v[i][0]*(V/w_v[i][1]) # 按照比例加入背包
            break
    return W

### 测试背包问题 

In [3]:
w = [20,30,10]
v = [60,120,50]
V = 50
print backpack(V,w,v)

16.6666666667


## 最小生成树算法
- Prim算法，顶点拓展  


1. 从图的顶点集合中任意选择一个顶点a作为初始节点，放入空集合T中，其余顶点放入集合G中
2. 如果G为空，则结束算法，否则转到3   
3. 从G中选择一个顶点x加入T中，顶点x与T中的一个节点y之间的边是集合G与集合T的连接边(边的顶点分别在两个集合中)中权重最小的  
4. 将x从G中删除  

### 定义 class graph 
- 使用邻接矩阵表示图(包括有向图与无向图)  
- 邻接矩阵是个方阵，记为$M_{nn}$,其中$M_{ij}$表示顶点$i$与顶点$j$之间是否存在一条边；具体来说，如果不是带有边权重的图，则$M_{ij} = 0$表示顶点$i$与顶点$j$之间不存在一条边，而$M_{ij} = 1$表示顶点$i$与顶点$j$之间存在一条边；如果是带有边权重的图，则$M_{ij} != 0$表示的是顶点$i$与顶点$j$之间边的权重
- 如果是无向图，则最终的邻接矩阵是对称的，否则就不一定

In [4]:
class graph:
    # 使用邻接矩阵便可以构造出图
    def __init__(self, g_matrix, is_directed=True, vertexs = None, edges = None):
        self.is_directed = is_directed # 是否是有向图
        self.vertexs = vertexs              # 顶点集合
        self.edges   = edges              # 边集合，使用(vertex_x,vertex_y)表示顶点x与y之间的一条边
        self.graph_matrix = g_matrix         # 邻接矩阵
    # 判断两个顶点之间是否有一条边
    def is_edges(self,vertex_x,vertex_y):
        return self.graph_matrix[vertex_x][vertex_y] != 0
    # 删除两个顶点之间的一条边
    def del_edges(self,vertex_x,vertex_y):
        if self.is_edges(vertex_x,vertex_y) == True:
            self.graph_matrix[vertex_x][vertex_y] = 0
            if self.is_directed == False:
                self.graph_matrix[vertex_y][vertex_x] = 0
            if self.edges != None and (vertex_x,vertex_y) in self.edges:
                self.edges.remove((vertex_x,vertex_y))
                if self.is_directed == False:
                    self.edges.remove((vertex_y,vertex_x))
                
    # 添加一条边
    def add_edges(self,vertex_x, vertex_y ,weight):
        self.graph_matrix[vertex_x][vertex_y] = weight
        if self.is_edges(vertex_x,vertex_y) == False:
            if self.is_directed == False:
                self.graph_matrix[vertex_y][vertex_x] = weight
            if self.edges != None and (vertex_x, vertex_y) not in self.edges:
                self.edges.append((vertex_x, vertex_y))
                if self.is_directed == False:
                    self.edges.append((vertex_y, vertex_x))
            


    # 生成顶点集合
    def build_vertexs(self):
        if self.vertexs == None:
            self.vertexs = [i for i in range(len(self.graph_matrix))]
    
    # 生成边集合
    def build_edges(self):
        if self.edges == None:
            self.edges = [] # 初始化一个空的边集合
            for i in range(len(self.graph_matrix)):
                for j in range(len(self.graph_matrix)):
                    if self.is_edges(i,j)==True:
                        self.edges.append((i,j))

## Prim算法 

In [5]:
def prim(g):
    """
    type g:graph
    rtype :graph
    """
    if g.vertexs == None:
        g.build_vertexs()
    init_vertex = 0
    T = [init_vertex]
    G = g.vertexs
    G.remove(init_vertex)


    mini_span_tree = graph([[0]*len(g.graph_matrix) for i in range(len(g.graph_matrix))],is_directed=False)

    while len(G) > 0:
        min_weight = float("inf")
        min_index_t = -1
        min_index_g = -1
        for i in T:  # 此处的时间复杂度可以通过牺牲空间来降低
            for j in G:
                if g.graph_matrix[i][j] > 0 and g.graph_matrix[i][j] < min_weight:
                    min_weight = g.graph_matrix[i][j]
                    min_index_t = i
                    min_index_g = j
        T.append(min_index_g)
        G.remove(min_index_g)
        mini_span_tree.add_edges(min_index_t, min_index_g,\
                                 g.graph_matrix[min_index_t][min_index_g]) # 作为最小生成树的一条边加入边集合
        mini_span_tree.add_edges(min_index_g, min_index_t, \
                                 g.graph_matrix[min_index_g][min_index_t])

    return mini_span_tree

### 测试prim算法

In [6]:
_ = [[0,3,0,0,6,5],[3,0,1,0,0,4],[0,1,0,6,0,4],[0,0,6,0,8,5],[6,0,0,8,0,2],\
           [5,4,4,5,2,0]]
_

[[0, 3, 0, 0, 6, 5],
 [3, 0, 1, 0, 0, 4],
 [0, 1, 0, 6, 0, 4],
 [0, 0, 6, 0, 8, 5],
 [6, 0, 0, 8, 0, 2],
 [5, 4, 4, 5, 2, 0]]

In [7]:
g_prim = graph(_,is_directed=False)
t_prim = prim(g_prim)

In [8]:
t_prim.graph_matrix

[[0, 3, 0, 0, 0, 0],
 [3, 0, 1, 0, 0, 4],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 0, 2],
 [0, 4, 0, 5, 2, 0]]

## 寻找单源最短路径
- Dijkstra 算法  
- 所谓单源最短路径，即是从图中的一个顶点a出发，到其余可达顶点的分别的最短路径  


1. 初始时定义两个顶点集合A={a},B={b,c,d,...},且a到B集合中顶点的路径长度为边权重或者无穷大(前一种情况对应邻接，后一种非邻接)，将对应的路径长度记录在route中
2. 判断B是否为空集，如果是则退出程序，否则转到3
3. 从B中选择一个顶点x加入A中，选择的标准(贪心策略)是dis(a,x)最小，且该值即使最终的结果,将x从B中删去，添加到A中;
4. 利用3中加入的x来重新更新route,具体的更新原则对于所有的 $y in B$ ,如果dis(a,y) > dis(a,x)+dis(x,y),则更新dis(a,y)为dis(a,x)+dis(x,y)

In [9]:
def find_shorted_route(init_vertex, g):
    if g.vertexs == None:
        g.build_vertexs()

    # 初始化顶点集合
    vertexs_a = [init_vertex]
    # g.build_vertexs()
    vertexs_b = g.vertexs
    vertexs_b.remove(init_vertex)

    # 初始化route
    route = {}
    for i in vertexs_b:
        if g.is_edges(init_vertex, i):
            route[i] = g.graph_matrix[init_vertex][i]
        else:
            route[i] = float("inf")  # 无穷大
    result = {}
    while len(vertexs_b) > 0:
        cur_vertex = min(route, key=lambda item: route[item])
        result[cur_vertex] = route[cur_vertex]
        vertexs_b.remove(cur_vertex)
        route.pop(cur_vertex)
        vertexs_a.append(cur_vertex)

        for key, length in route.iteritems():
            if g.graph_matrix[cur_vertex][key]>0 and length > result[cur_vertex] + g.graph_matrix[cur_vertex][key]:
                route[key] = result[cur_vertex] + g.graph_matrix[cur_vertex][key]
    return result

### 测试迪杰斯特拉算法

In [10]:
temp_graph = [[0,3,0,7,0],[3,0,4,2,0],[0,4,0,5,6],[7,2,5,0,4],[0,0,6,4,0]]
g_a = graph(temp_graph, is_directed=True)

In [11]:
find_shorted_route(init_vertex=0,g=g_a)

{1: 3, 2: 7, 3: 5, 4: 9}